In [8]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from net import Net

In [9]:
# Use CUDA if possible
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [10]:
train_dir = './train_images'    # folder containing training images
test_dir = './test_images'    # folder containing test images

transform = transforms.Compose(
    [transforms.Grayscale(),   # transforms to gray-scale (1 input channel)
     transforms.ToTensor(),    # transforms to Torch tensor (needed for PyTorch)
     transforms.Normalize(mean=(0.5,),std=(0.5,))]) # subtracts mean (0.5) and devides by standard deviation (0.5) -> resulting values in (-1, +1)

# Define the transformations for augmentation
transforms_augmented = {
    'train': transforms.Compose([
        transforms.Grayscale(), 
        transforms.RandomResizedCrop(36),
        transforms.RandomHorizontalFlip(0.25),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,),std=(0.5,))
    ]),
    'test':transforms.Compose([
        transforms.Grayscale(), 
        transforms.ToTensor(), 
        transforms.Normalize(mean=(0.5,),std=(0.5,))])        
}

In [11]:
# Define two pytorch datasets (train/test) 
train_data_augmented = torchvision.datasets.ImageFolder(train_dir, transform=transforms_augmented['train'])
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

valid_size = 0.2   # proportion of validation set (80% train, 20% validation)
batch_size = 128

# Define randomly the indices of examples to use for training and for validation
num_train = len(train_data_augmented)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

# Define two "samplers" that will randomly pick examples from the training and validation set
train_sampler = SubsetRandomSampler(train_new_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [12]:
# Dataloaders (take care of loading the data from disk, batch by batch, during training)
train_loader = torch.utils.data.DataLoader(train_data_augmented, batch_size=batch_size, sampler=train_sampler, num_workers=4)
valid_loader = torch.utils.data.DataLoader(train_data_augmented, batch_size=batch_size, sampler=valid_sampler, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4)

classes = ('noface','face')  # indicates that "1" means "face" and "0" non-face (only used for display)


In [13]:
net = Net()
net = net.to(device)
n_epochs = 64

optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [14]:
# Training 
running_loss =0
# loop over epochs: one epoch = one pass through the whole training dataset
for epoch in range(1, n_epochs+1):  
#   loop over iterations: one iteration = 1 batch of examples
    running_loss =0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # zero the gradient buffers
        output = net(data)
        loss = criterion(output, target)
        running_loss +=loss
        loss.backward()
        optimizer.step() # Does the update
    print ('epoch: %d, running_loss: %5.7f' % (epoch,running_loss))  

epoch: 1, running_loss: 233.0632935
epoch: 2, running_loss: 158.7202454
epoch: 3, running_loss: 126.3238373
epoch: 4, running_loss: 104.8042374
epoch: 5, running_loss: 92.1080246
epoch: 6, running_loss: 81.6177216
epoch: 7, running_loss: 77.5640030
epoch: 8, running_loss: 72.5069427
epoch: 9, running_loss: 67.0175323
epoch: 10, running_loss: 65.3389130
epoch: 11, running_loss: 62.1663666
epoch: 12, running_loss: 59.0074997
epoch: 13, running_loss: 57.4701843
epoch: 14, running_loss: 52.4946747
epoch: 15, running_loss: 55.2193222
epoch: 16, running_loss: 54.0557480
epoch: 17, running_loss: 52.0162621
epoch: 18, running_loss: 49.1355820
epoch: 19, running_loss: 50.1782341
epoch: 20, running_loss: 47.5283928
epoch: 21, running_loss: 47.3824615
epoch: 22, running_loss: 45.9070282
epoch: 23, running_loss: 45.7562408
epoch: 24, running_loss: 43.8760719
epoch: 25, running_loss: 43.5629616
epoch: 26, running_loss: 43.0898781
epoch: 27, running_loss: 43.7409859
epoch: 28, running_loss: 42.36983

In [15]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %5.6f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 97.954903 %
